## Get First Run Date Range to Copy GMail Messages to AWS s3

In [1]:
import boto3
import pickle


def get_creds():
    sm_client = boto3.client(
        'secretsmanager',
        region_name='us-east-1'
    )

    secret_token = sm_client.get_secret_value(SecretId='gmail_token')['SecretBinary']
    creds = pickle.loads(secret_token)
    return creds

In [2]:
from googleapiclient.discovery import build

creds = get_creds()
service = build('gmail', 'v1', credentials=creds)

In [3]:
users = service.users()

In [4]:
def get_job_details(job_name):
    dynamodb = boto3.resource('dynamodb')
    table = dynamodb.Table('gmail_jobs')
    job_details = table.get_item(Key={'job_id': job_name})['Item']
    return job_details

In [5]:
job_details = get_job_details('gmail_ingest')
job_details

{'job_description': 'Ingest data from gmail to s3',
 'is_active': 'Y',
 'job_id': 'gmail_ingest',
 'baseline_days': Decimal('45'),
 'job_run_bookmark_details': {'last_run_max_message_id': '180b9754d2b3617f',
  'last_run_start_time_epoch': Decimal('1652293800'),
  'last_run_end_time_epoch': Decimal('1652380200')}}

In [6]:
import datetime

In [7]:
datetime.timedelta?

Init signature: datetime.timedelta(self, /, *args, **kwargs)
Docstring:     
Difference between two datetime values.

timedelta(days=0, seconds=0, microseconds=0, milliseconds=0, minutes=0, hours=0, weeks=0)

All arguments are optional and default to 0.
Arguments may be integers or floats, and may be positive or negative.
File:           /usr/local/Cellar/python@3.9/3.9.12/Frameworks/Python.framework/Versions/3.9/lib/python3.9/datetime.py
Type:           type
Subclasses:     


In [8]:
datetime.datetime.now().date()

datetime.date(2022, 5, 14)

In [9]:
datetime.datetime.now().date() - datetime.timedelta(days=45)

datetime.date(2022, 3, 30)

In [10]:
baseline_days = int(job_details['baseline_days'])
baseline_days

45

In [11]:
start_time = datetime.datetime.now().date() - datetime.timedelta(days=int(baseline_days))
start_time

datetime.date(2022, 3, 30)

In [12]:
end_time = start_time + datetime.timedelta(days=1)
end_time

datetime.date(2022, 3, 31)

In [13]:
import time

In [14]:
start_time_epoch = int(time.mktime(start_time.timetuple()))
start_time_epoch

1648578600

In [15]:
end_time_epoch = int(time.mktime(end_time.timetuple()))   
end_time_epoch

1648665000

In [16]:
messages = users. \
    messages(). \
    list(
        userId='me', 
        q=f'after:{start_time_epoch} before:{end_time_epoch}'
    ). \
    execute()

In [17]:
messages

{'messages': [{'id': '17fdc0b349836c2f', 'threadId': '17fdc0b349836c2f'},
  {'id': '17fdc0139bb6ef0e', 'threadId': '17fdc0139bb6ef0e'},
  {'id': '17fdbf69c6a845ea', 'threadId': '17fdbf69c6a845ea'},
  {'id': '17fdbd3a5bbc0550', 'threadId': '17fdbd3a5bbc0550'},
  {'id': '17fdbd0a3a385f4c', 'threadId': '17fdbd0a3a385f4c'},
  {'id': '17fdbc81ba38885c', 'threadId': '17fdbc81ba38885c'},
  {'id': '17fdbb8fdc3029e0', 'threadId': '17fdbb8fdc3029e0'},
  {'id': '17fdbb202cec0aec', 'threadId': '17fdbb202cec0aec'},
  {'id': '17fdba6487207633', 'threadId': '17fdba6487207633'},
  {'id': '17fdb9f619d8c36d', 'threadId': '17fdb9f619d8c36d'},
  {'id': '17fdb9ad57f34dbe', 'threadId': '17fdb9ad57f34dbe'},
  {'id': '17fdb8185604211e', 'threadId': '17fdb8185604211e'},
  {'id': '17fdb75066750fce', 'threadId': '17fdb75066750fce'},
  {'id': '17fdb64366fad0c4', 'threadId': '17fdb64366fad0c4'},
  {'id': '17fdb4f1cd3f25a7', 'threadId': '17fdb4f1cd3f25a7'},
  {'id': '17fdb4c53bf4c77b', 'threadId': '17fdb4c53bf4c77b